# First Simulation for CausalMARL

## TODO
1. NN paras
2. why IS和DR相比variance差不多，我觉得有点奇怪 -CV?
1. output: convenient for R
1. no_MF so large? why?

print ATE and others

## WAIT
2. 如何体现DR？
3. 使得overfitting? 把biad 弄下去？
1. 之前一直sd小 所以看不出趋势; 现在大了 算力不够
1. 还是depends on the pattern
1. why variance 差别这么小?

1. 找setting
    1. T 
    2. sd_R
    3. 高太多？
    4. 看之前的结果 理解trend
    5. simple 会怎样？
    6. less bias, large variance?
        1. 2. T=48*7=336的？就相当于一个星期，这样variance大一点
        4. setting里e_{t,g}^R可以稍微调大一点，让sd大一些
1. NN random number
1. max_iteration = 1001, 故意弄坏IS？ but still no trend?

QV CV 会有什么影响？

why no trend with sd_R? 
bias increase
std decrease?
I tries with smaller?

1. Time cost: 96 cores for 96 reps. 2 min for 1 region (17s for one NN, and CV for QV). 200 mins = 3h for per setting (4 targets). 

## TODO
3. 整理代码 - 他的需求
4. 汇报结果
    1. 画图
    2. 整理setting
1. draft

正在实验 看看sd_R

1. 96 replications, 25regions. 4 targets (maybe more) = 3 hours
2. 7-14 = 8 settings (no clear trend?). patterns

## Objective
1. value estimations are all good
2. difference estimations are all good
3. better than IS in 1, 2
4. better than single agent (in bias) in 1, 2
5. better than all neigh infomation (in variance) in 1, 2


1. ours
2. average
3. QV 
4. DR_no_MARL
5. DR_no_Mean_Field

## Tuning
1. Noise
    1. sd_O
    2. sd_R
    3. sd_D
2. spatial
    1. w_O
    2. w_A
2. hyperparameters of NN and Q/V
2. Region difference
    1. sd_u_O
    2. u_D = np.mean(u_O) - 2
5. T
6. l
7. threshold_range
8. pattern seed
10. n_neigh (simple = False)
1. Archive
    1. mean_reversion: D_t = (D_t + u_D) / 2
    9. p_behav
    4. reward def

# Setting
### Model
0. Mis
    1. Size: grid, N = 5 * 5, T = 48 * 14
    2. at time t, based on $O_t$ and $D_t$ which are observations  during (t-1, t], we (randomly) choose action $A_t$ which will be implemented during (t, t + 1], and observe the reward $R_t$ which is defind as the mismatch during (t, t + 1].
1. State:
    1. Order: 
        1. $u^O_{l} \sim logN(2.5, .3)$  (mean = 12.5, std = 3)
        2. $O_{l,t} \sim u^O_{l} + e^O_{l,t}, e^O_{l,t} \sim (Poisson(1) - 1)$
    2. Driver: 
        1. Attraction of $l$: $Att_{l,t} = exp(w_A * A_{l,t}) + w_O * (\frac{O_{l,t}}{1 + D_{l,t}})$
        2. Flow:  $D_{l, t + 1} = \sum_{i \in N_l} (\frac{Att_{l,t}}{\sum_{j \in N_j}Att_{j,t}}D_{i,t} )$
        3. Computation: matrix multiplication
    4. Mismatch: $M_{t+1,g} = 0.5 * (1-\frac{|D_{t+1,g}-O_{t+1,g}|}{|1 + D_{t+1,g}+O_{t+1,g}|}) + 0.5 * M_{t,g}$
4. Reward: $R_{t,g} = M_{t + 1, g}min(D_{t + 1,g}, O_{t + 1,g}) + e^R_{t,g}$ 
3. Behav policy: $A_{t,g} \sim Bernoulli(0.5)$
5. Target policy:  $A_{t,g} = I_{\{u^O_l \ge 12\}}$

3. Competing methods: 
    1. "IS": Importance Sampling (Lihong Li) + mean-field
    2. "Susan": Value funcions (Susan) + mean-field
    3. "DR_NS": DR without considerinig other agents
    2. DR w/o mean-field: $R_it$ with neigh states/actions (multi-dim actions)

8. Report results for one $\{u_{o,g}\}_g$ pattern with four potential target policies.

## Archive
2. $\vec{D}_{t+1} = \frac{1}{2}(\omega_t^{-1} (\mathbf{O}_t \times (\mathbf{A}_{t} + \mathbf{I}) \times \mathbf{Adj}  \times (\mathbf{O}_t + \mathbf{I})^{-1})  \vec{D}_{t} + 10) + \vec{\epsilon}^D_{t+1}$
3. $\vec{D}_{t+1} = \frac{1}{2}(\omega_t^{-1} (\mathbf{O}_t \times (\mathbf{A}_{t} + \mathbf{I}) \times \mathbf{Adj}  \times (\mathbf{O}_t + \mathbf{I})^{-1})  \vec{D}_{t} + 10 * (2 - sin(t/48 * 2\pi))) + \vec{\epsilon}^D_{t+1}$
2. $O_{t,g} \sim Poisson(10 * (2 - sin(t/48 * 2\pi)))$ [no heterogeneity] -> mean is 20?
2. $R_{i,t}=M_{i,t} * min(D_{i,t}, O_{i,t})+mean \{M_{j,t} * min(D_{j,t}, O_{j,t})\}_{j\in N(i)}+\epsilon_{i,t}$
1. $A_{t,g} = a_g, a_g \sim Bernoulli(0.5)$
2. $M_{t+1,g} = 1-\frac{|D_{t+1,g}-O_{t+1,g}|}{|D_{t+1,g}+O_{t+1,g}|}$
2. $O_{t,g} \sim Poisson(10)$
1. $\vec{D}_{t+1} = \omega_t^{-1} \mathbb{P}(((w_O * \mathbf{O}_t + \mathbf{I}) \times (w_A * \mathbf{A}_{t} +  \mathbf{I}) \times \mathbf{Adj}  \times (w_O * \mathbf{O}_t + \mathbf{I})^{-1})  \vec{D}_{t} + \vec{\epsilon}^D_{t+1})$
    1. $w_t$ is a normalization parameter, $\mathbf{A}_{t}$ and $\mathbf{O}_t$ are corresponding diagonal matries, and $\vec{\epsilon}^D_t \sim (Poisson(1) - 1)$
    2. operator $\mathbb{P}$ is defined as $\mathbb{P}(\vec{D}) = (max(int(D_1),1),\dots, max(int(D_N),1))^T$      
1. + \epsilon^R_{t,g}, \epsilon^R_{t,g} \sim N(0,sd_R)$


### Metrics and competing methods
1. Ground truth: MC-based proxy
2. Metrics: bias, std and MSE (100 replicates)